In [3]:
import pandas as pd
from datetime import datetime as dt
from IPython.display import display

In [4]:
# čitanje podataka iz .csv datoteke "DatasetHandballMatches"
location = "C:/Users/Josipa Kaselj/FER/Diplomski/4.godina/2.semestar/SEMINAR2/"
raw_data_1 = pd.read_csv(location + "1st_dataset.csv")
raw_data_2 = pd.read_csv(location + "2nd_dataset.csv")
raw_data_3 = pd.read_csv(location + "3rd_dataset.csv")
raw_data_4 = pd.read_csv(location + "4th_dataset.csv")

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%y').date()

raw_data_1.Date = raw_data_1.Date.apply(parse_date)
raw_data_2.Date = raw_data_2.Date.apply(parse_date)
raw_data_3.Date = raw_data_3.Date.apply(parse_date)
raw_data_4.Date = raw_data_4.Date.apply(parse_date)

# u datasetu se osim značajki koje utječu na konačan rezultat utakmice i predikciju ishoda istih, nalaze i drugi podatci koji ne utječi izravno na rezultat predikcije

# dohvaćanje samo ključnih značajki
important_features = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

match_statistics_1 = raw_data_1[important_features]
match_statistics_2 = raw_data_2[important_features]
match_statistics_3 = raw_data_3[important_features]
match_statistics_4 = raw_data_4[important_features]

#display(match_statistics_1)

In [5]:
# Dohvaćanje ukupnog broja postignutih golova do tada, do trenutnog tjedna
def get_scored_goals(match_statistics):
    # Stvaranje dictionary s nazivima timova kao ključevima
    teams = {}
    for i in match_statistics.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    #print(len(teams))

    for i in range(0, len(match_statistics)):
        HTGS = match_statistics.iloc[i]['FTHG']
        ATGS = match_statistics.iloc[i]['FTAG']
        #print(match_statistics.iloc[i].HomeTeam)
        teams[match_statistics.iloc[i].HomeTeam].append(HTGS)
        teams[match_statistics.iloc[i].AwayTeam].append(ATGS)

    # Stvara se dataframe za postignute golove gdje su retci timovi, a stupci su utakmice u kojima je svaki tim sudjelovao
    ScoredGoals = pd.DataFrame(data=teams, index = [i for i in range(1,31)]).T
    ScoredGoals[0] = 0

    for i in range(2,31):
        ScoredGoals[i] = ScoredGoals[i] + ScoredGoals[i-1]
    return ScoredGoals


# Dohvaćanje ukupnog broja primljenih golova do tada, do trenutnog tjedna
def get_conceded_goals(match_statistics):
    # Stvaranje dictionary s nazivima timova kao ključevima
    teams = {}
    for i in match_statistics.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    for i in range(0, len(match_statistics)):
        HTGC = match_statistics.iloc[i]['FTHG']
        ATGC = match_statistics.iloc[i]['FTAG']
        teams[match_statistics.iloc[i].HomeTeam].append(HTGC)
        teams[match_statistics.iloc[i].AwayTeam].append(ATGC)

    # Stvara se dataframe za primljene golove gdje su retci timovi, a stupci su utakmice u kojima je svaki tim sudjelovao
    ConcededGoals = pd.DataFrame(data=teams, index = [i for i in range(1,31)]).T
    ConcededGoals[0] = 0

    for i in range(2,31):
        ConcededGoals[i] = ConcededGoals[i] + ConcededGoals[i-1]
    return ConcededGoals


def get_gss(match_statistics):
    GS = get_scored_goals(match_statistics)
    GC = get_conceded_goals(match_statistics)
    #print(GS)

    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []
    j = 0

    for i in range(240):
        ht = match_statistics.iloc[i].HomeTeam
        at = match_statistics.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])

        if ((i+1) % 8) == 0:
            j = j + +1

    match_statistics['HTGS'] = HTGS
    match_statistics['ATGS'] = ATGS
    match_statistics['HTGC'] = HTGC
    match_statistics['ATGC'] = ATGC

    return match_statistics

#print(get_scored_goals(match_statistics))
#print()
#print(get_conceded_goals(match_statistics))

match_statistics_1 = get_gss(match_statistics_1)
match_statistics_2 = get_gss(match_statistics_2)
match_statistics_3 = get_gss(match_statistics_3)
match_statistics_4 = get_gss(match_statistics_4)

#print(match_statistics_1)

C:\Users\Josipa Kaselj\AppData\Local\Temp\ipykernel_23144\230420341.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_statistics['HTGS'] = HTGS
C:\Users\Josipa Kaselj\AppData\Local\Temp\ipykernel_23144\230420341.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_statistics['ATGS'] = ATGS
C:\Users\Josipa Kaselj\AppData\Local\Temp\ipykernel_23144\230420341.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [6]:
def get_points(result):
    if result == 'W':
        return 2
    elif result == 'D':
        return 1
    else:
        return 0

def get_cuml_points(match_result):
    match_result_points = match_result.applymap(get_points)
    for i in range(2,31):
        match_result_points[i] = match_result_points[i] + match_result_points[i-1]

    match_result_points.insert(column =0, loc = 0, value = [0*i for i in range(16)])
    return match_result_points

def get_match_result(match_statistics):
    # Stvaranje dictionary s nazivima timova kao ključevima
    teams = {}
    for i in match_statistics.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    for i in range(len(match_statistics)):
        if match_statistics.iloc[i].FTR == 'H':
            teams[match_statistics.iloc[i].HomeTeam].append('W')
            teams[match_statistics.iloc[i].AwayTeam].append('L')
        elif match_statistics.iloc[i].FTR == 'A':
            teams[match_statistics.iloc[i].AwayTeam].append('W')
            teams[match_statistics.iloc[i].HomeTeam].append('L')
        else:
            teams[match_statistics.iloc[i].AwayTeam].append('D')
            teams[match_statistics.iloc[i].HomeTeam].append('D')

    return pd.DataFrame(data=teams, index = [i for i in range(1,31)]).T


def get_aggregate_points(match_statistics):
    match_result = get_match_result(match_statistics)
    cum_pts = get_cuml_points(match_result)
    HTP = []
    ATP = []
    j = 0

    for i in range(240):
        ht = match_statistics.iloc[i].HomeTeam
        at = match_statistics.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i+1) % 8) == 0:
            j = j + +1

    match_statistics['HTP'] = HTP
    match_statistics['ATP'] = ATP
    return match_statistics

match_statistics_1 = get_aggregate_points(match_statistics_1)
match_statistics_2 = get_aggregate_points(match_statistics_2)
match_statistics_3 = get_aggregate_points(match_statistics_3)
match_statistics_4 = get_aggregate_points(match_statistics_4)

#print(match_statistics_1)

In [7]:
def get_form(match_statistics, num):
    form = get_match_result(match_statistics)
    form_final = form.copy()
    for i in range(num,31):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

def add_form(match_statistics, num):
    #print("num: ", num)
    form = get_form(match_statistics,num)
    h = ['M' for i in range(num * 8)]
    a = ['M' for i in range(num * 8)]
    #print(h)
    #print(a)
    j = num

    for i in range((num*8),240):
        ht = match_statistics.iloc[i].HomeTeam
        at = match_statistics.iloc[i].AwayTeam

        past = form.loc[ht][j]
        h.append(past[num-1])

        past = form.loc[at][j]
        a.append(past[num-1])

        if ((i+1) % 8) == 0:
            j = j + +1

    match_statistics['HM' + str(num)] = h
    match_statistics['AM' + str(num)] = a
    return match_statistics

def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics

match_statistics_1 = add_form_df(match_statistics_1)
match_statistics_2 = add_form_df(match_statistics_2)
match_statistics_3 = add_form_df(match_statistics_3)
match_statistics_4 = add_form_df(match_statistics_4)

#print(match_statistics_1)

In [8]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

match_statistics_1 = match_statistics_1[cols]
match_statistics_2 = match_statistics_2[cols]
match_statistics_3 = match_statistics_3[cols]
match_statistics_4 = match_statistics_4[cols]

In [9]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(240):
        MatchWeek.append(j)
        if ((i + 1)% 8) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

match_statistics_1 = get_mw(match_statistics_1)
match_statistics_1 = match_statistics_1[match_statistics_1.MW > 3]
match_statistics_2 = get_mw(match_statistics_2)
match_statistics_2 = match_statistics_2[match_statistics_2.MW > 3]
match_statistics_3 = get_mw(match_statistics_3)
match_statistics_3 = match_statistics_3[match_statistics_3.MW > 3]
match_statistics_4 = get_mw(match_statistics_4)
match_statistics_4 = match_statistics_4[match_statistics_4.MW > 3]
#print(match_statistics_1)

In [10]:
match_statistics = pd.concat([match_statistics_1,
                              match_statistics_2,
                              match_statistics_3,
                              match_statistics_4], ignore_index=True)

n_matches = match_statistics.shape[0]

# Izračun broja utakmica u kojima je pobjednik 'HomeTeam', 'AwayTeam' ili je neriješen rezultat.
n_homewins = len(match_statistics[match_statistics.FTR == 'H'])
n_awaywins = len(match_statistics[match_statistics.FTR == 'A'])
n_draws = len(match_statistics[match_statistics.FTR == 'D'])


# Izračun postotka za pobjede 'HomeTeam' i 'AwayTeam' te neriješene utakmice.
home_win_rate = (float(n_homewins) / (n_matches)) * 100
away_win_rate = (float(n_awaywins) / (n_matches)) * 100
draw_rate = (float(n_draws) / (n_matches)) * 100

# Ispis rezultata postotaka.
print("Total number of matches: {}".format(n_matches))
print("Number of features: {}".format(match_statistics.shape[1] - 1))
print("Number of matches won by home team: {}".format(n_homewins))
print("Win rate of home team: {:.2f}%".format(home_win_rate))
print()
print("Number of matches won by away team: {}".format(n_awaywins))
print("Win rate of away team: {:.2f}%".format(away_win_rate))
print()
print("Number of draw matches: {}".format(n_draws))
print("Draw rate: {:.2f}%".format(draw_rate))

# Dohvaćanje bodova za forme timova
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

match_statistics['HTFormPtsStr'] = match_statistics['HM1'] + match_statistics['HM2'] + match_statistics['HM3'] + match_statistics['HM4'] + match_statistics['HM5']
match_statistics['ATFormPtsStr'] = match_statistics['AM1'] + match_statistics['AM2'] + match_statistics['AM3'] + match_statistics['AM4'] + match_statistics['AM5']

match_statistics['HTFormPts'] = match_statistics['HTFormPtsStr'].apply(get_form_points)
match_statistics['ATFormPts'] = match_statistics['ATFormPtsStr'].apply(get_form_points)

# Identificiranje nizova pobjeda i poraza ako postoje.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0

def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0

def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0

def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0

match_statistics['HTWinStreak3'] = match_statistics['HTFormPtsStr'].apply(get_3game_ws)
match_statistics['HTWinStreak5'] = match_statistics['HTFormPtsStr'].apply(get_5game_ws)
match_statistics['HTLossStreak3'] = match_statistics['HTFormPtsStr'].apply(get_3game_ls)
match_statistics['HTLossStreak5'] = match_statistics['HTFormPtsStr'].apply(get_5game_ls)

match_statistics['ATWinStreak3'] = match_statistics['ATFormPtsStr'].apply(get_3game_ws)
match_statistics['ATWinStreak5'] = match_statistics['ATFormPtsStr'].apply(get_5game_ws)
match_statistics['ATLossStreak3'] = match_statistics['ATFormPtsStr'].apply(get_3game_ls)
match_statistics['ATLossStreak5'] = match_statistics['ATFormPtsStr'].apply(get_5game_ls)

match_statistics.keys()

Total number of matches: 864
Number of features: 22
Number of matches won by home team: 408
Win rate of home team: 47.22%

Number of matches won by away team: 313
Win rate of away team: 36.23%

Number of draw matches: 143
Draw rate: 16.55%


Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'MW', 'HTFormPtsStr', 'ATFormPtsStr',
       'HTFormPts', 'ATFormPts', 'HTWinStreak3', 'HTWinStreak5',
       'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3', 'ATWinStreak5',
       'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [12]:
# Izračun razlike postignutih i primljenih golova za 'HomeTeam' i 'AwayTeam'
match_statistics['HTGD'] = match_statistics['HTGS'] - match_statistics['HTGC']
match_statistics['ATGD'] = match_statistics['ATGS'] - match_statistics['ATGC']

# Izračun razlike u bodovima između timova koji susdjeluju u utakmici
match_statistics['DiffPts'] = match_statistics['HTP'] - match_statistics['ATP']
match_statistics['DiffFormPts'] = match_statistics['HTFormPts'] - match_statistics['ATFormPts']

In [14]:
# Skaliranje DiffPts, DiffFormPts, HTGD i ATGD s brojem utakmica koje je svaki tim odigrao do tada.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
match_statistics.MW = match_statistics.MW.astype(float)

for col in cols:
    match_statistics[col] = match_statistics[col] / match_statistics.MW

In [15]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'

match_statistics['FTR'] = match_statistics.FTR.apply(only_hw)
#print(len(match_statistics))
match_statistics_test = match_statistics[300:420]

In [16]:
match_statistics.to_csv(location + "final_dataset.csv")
match_statistics_test.to_csv(location + "test_dataset.csv")